# Download selected variables from the INGRID CMIP6 database
* Organize data by variable/model and save each individual year into a separate file. Easiest way, I think, to more uniformly handle things because of loack of standardization in datafiles.
* Database here: http://mary.ldeo.columbia.edu:81/CMIP6/
* For master list of everything available: wget http://mary.ldeo.columbia.edu/master_collection/mary_cmip6.csv

## Start Clean

In [1]:
# Reset the environment (start clean)
%reset -f

# Import Modules and define functions
import calendar
import datetime
import os
import numpy as np
import numpy.ma as ma
import netCDF4
import matplotlib
import copy
from matplotlib import pyplot as plt
import scipy
import scipy.signal
#import scipy.io as sio
#import seaborn as sns
#import pandas as pd
import scipy.stats as stats
#import statsmodels.api as sm
from IPython.display import display
import requests
import pandas as pd
import os

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [31]:
all_models=False
if all_models:
# Model Names
    model_names=(    'AWI-CM-1-1-MR',	
                     'BCC-CSM2-MR',		
                     'BCC-ESM1',
                     'CAMS-CSM1-0',      # NEED TO REDO
                     'CanESM5',			
                     'CESM2',			
                     'CESM2-WACCM',		
                     'CNRM-CM6-1',		
                     'CNRM-ESM2-1',		
                     'E3SM-1-0',		
                     'EC-Earth3',       # NEED TO REDO
                     'EC-Earth3-LR',    # NEED TO REDO
                     'EC-Earth3-Veg',   # NEED TO REDO
                     'FGOALS-f3-L',     # NEED TO REDO
                     'GFDL-AM4', 
                     'GFDL-CM4',  # PI CONTROL PROBLEMS		
                     'GFDL-ESM4',		
                     'GISS-E2-1-G',		
                     'GISS-E2-1-H',		
                     'HadGEM3-GC31-LL',
                     'IPSL-CM6A-LR',	 
                     'MIROC6',			# mrsos and tas done
                     'MIROC-ES2L',
                     'MPI-ESM1-2-HR',   
                     'MRI-ESM2-0',       # mrsos and tas done
                     'NESM3',             # mrsos and tas done
                     'SAM0-UNICON',		# mrsos and tas done
                     'UKESM1-0-LL',	    # mrsos and tas done
                  )
else:
    model_names=['CESM2']
    #model_names=('IPSL-CM6A-LR',
     #           'CESM2')
for x in enumerate(model_names):
    print(x)

(0, 'CESM2')


## Setup: Choose models/ensemble members/variables/etc

In [36]:
#Month array
mons=np.arange(12)+1

# Baseline directory
base_dir   = 'http://mary.ldeo.columbia.edu:81/CMIP6/.'

# Output directory
out_dir = '/Users/kmarvel/Documents/DATA/LGM/FromMary/'

# MIP/Sim Name (WITH YEAR RANGES)
#experiment_id = ('piControl')   # historical
experiment_id = ('historical')   # historical
#experiment_id = ('ssp245')       # RCP 4.5 equivalent
#experiment_id = ('ssp585')       # RCP 8.5 equivalent
#experiment_id = ('ssp370')       # RCP 7.0 equivalent
#experiment_id = ('abrupt-4xCO2')    
# Variable info: name, category, units_old, units_new, scale for new unites 
# var_info = np.array([ ['rsutcs','W m-2', 'W m-2',1, 'TOA Clear-sky Outgoing Shortwave Radiation'],\
#                      ['rsut', 'W m-2', 'W m-2', 1, 'TOA Outgoing Shortwave Radiation'],\
#                      ['rsdt', 'W m-2', 'W m-2', 1, 'TOA Downwelling Shortwave Radiation'],\
#                      ['rlut','W m-2', 'W m-2',1, 'TOA Outgoing Longwave Radiation'],\
#                      ['rlutcs','W m-2', 'W m-2',1, 'TOA Clear-sky Outgoing Longwave Radiation'],\
#                      ['ts','K','K',1,'surface air temperature'],\
#                      ])


var_info=np.array([['rsdt', 'W m-2', 'W m-2', 1, 'TOA Downwelling Shortwave Radiation']])
#var_info = np.array([['clt', '%', '%',1,'Total Cloud fraction']])
#var_info = np.array([   ['rlut','W m-2','W m-2',1,'TOA Outgoing Longwave Radiation'],  
           # ])

#var_info = np.array([  ['evspsbl', 'kg m-2 s-1', 'mm day-1',86400, 'evaporation rate, including transpiration and sublimation'],
#
#            ])

# Load dataframe containing all information available
ingrid_cmip6 = pd.read_csv('mary_cmip6_101419.csv')
#member_id="r1i1p1f1"
member_id="r2i1p1f1"
print(var_info)
ingrid_cmip6

# Initialize master list to use for post-processing
df_proclist = pd.DataFrame(columns=['model','sim','ensemble','variable'])

# just some debugging and testing
#df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod) & (ingrid_cmip6.variable_id==curr_var) & (ingrid_cmip6.experiment_id == experiment_id)]
#df1 = ingrid_cmip6[(ingrid_cmip6.source_id=='BCC-CSM2-MR') & (ingrid_cmip6.variable_id=='lai')]
#df1


[['rsdt' 'W m-2' 'W m-2' '1' 'TOA Downwelling Shortwave Radiation']]


## For Accessing Directory CMIP6 (NOT CMIP6i)
* Grab everything available. Put the years in the filenames.
* also grabbed fixed variables, if available: sftlf, areacella

In [37]:
# Nested model/variable loop
for n_mod in enumerate(model_names):

    # Variable/model Info
    curr_mod     = n_mod[1]
    
    print(curr_mod)
    # Variables Loop--------------------------------------------------------------------------------------------------------------------------------------------------------------
    for n_var in enumerate(var_info):
        
        curr_var     = n_var[1][0]
        print(curr_var)
        curr_units   = n_var[1][2]
        curr_scale   = np.float(n_var[1][3])
        curr_name    = n_var[1][4]
    
        # search for all versions for one model
        if member_id is None:
            df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod)\
                               & (ingrid_cmip6.variable_id==curr_var) & \
                               (ingrid_cmip6.experiment_id == experiment_id)]
            num_ens=df1.shape[0]
            print("number of ensembles = "+str(num_ens))
        else:
            df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod)\
                               & (ingrid_cmip6.variable_id==curr_var) & \
                               (ingrid_cmip6.experiment_id == experiment_id)&\
                              (ingrid_cmip6.member_id == member_id)]
            num_ens=1
        # OLD Drop duplicate ensemble members (split by year)
        #df1.drop_duplicates(subset ="member_id",keep = False, inplace = True) 

        # Number of ensemble members available
        
        
        # Ensemble Loop----------------------------------------------------------------------------------------------------------------------------------------------------------
        for i_ens in np.arange(num_ens):

            # Construct Remote OpenDAP Link
            nc_link = base_dir+df1.activity_id.iloc[i_ens]+'/.'+df1.institution_id.iloc[i_ens]+'/.'+curr_mod+'/.'+experiment_id+'/.'+df1.member_id.iloc[i_ens]+'/.'+df1.table_id.iloc[i_ens]+ \
                        '/.'+curr_var+'/.'+df1.grid_label.iloc[i_ens]+'/.'+df1.version.iloc[i_ens]+'/.'+df1.file_basename.iloc[i_ens]+'/.'+curr_var+'/dods'

            print(curr_mod+':'+experiment_id+':'+df1.member_id.iloc[i_ens]+':'+curr_var)
            df_proclist.append(pd.Series([curr_mod,experiment_id,df1.member_id.iloc[i_ens],curr_var],index=df_proclist.columns),ignore_index=True)
            
            # Check if link exists
            request = requests.get(nc_link)    
            
            # If this Exists, then load the data
            if request.status_code==200:
                #print(nc_link)       
        
                # Open the remote file-----------------------------------------------------------------------------------------------------------------------------------------

                # Create a Remote Link
                nc_file = netCDF4.Dataset(nc_link) 
    
                # Pull out variables 
                var_nc = nc_file.variables[curr_var]
                lat    = nc_file.variables['lat'][:]
                lon    = nc_file.variables['lon'][:]

                # Time Stuff
                nctime    = nc_file.variables['time'][:]          # time data
                unit_time = nc_file.variables['time'].units       # units
                cal_time  = nc_file.variables['time'].calendar    # calendar 

                # Loop through and pull out year and month vectors from datetime object. 
                # For picontrol, 
#                 if len(nctime)>200:
#                     nctime=nctime[:200]
                yr_vect  = np.zeros(np.shape(nctime))
                
                mon_vect = np.zeros(np.shape(nctime))

                for i_date in enumerate(nctime):
                    yr_vect[i_date[0]]  = netCDF4.num2date(nctime[i_date[0]],units=unit_time,calendar=cal_time).year
                    mon_vect[i_date[0]] = netCDF4.num2date(nctime[i_date[0]],units=unit_time,calendar=cal_time).month

                # Year Vector (one value per year)
                yrs     = np.unique(yr_vect)

                # -------------------------------------------------------------------------------------------------------------------------------------------------------------
                # Pull out the data year by year and save in individual year files
                
                # Make model directory if it does not exist
                if os.path.isdir(out_dir+'/'+curr_var+'/'+curr_mod)==False:
                    os.makedirs(out_dir+'/'+curr_var+'/'+curr_mod)
                
                # Year Loop
                for i_yr in enumerate(yrs):
                    # Locate all months, current year
                    loc_yr = np.where(yr_vect==i_yr[1])[0]
    
                    # Pull out current year of data and put in the storage array
                    var_data = copy.deepcopy(var_nc[loc_yr,:,:])
    
                    # Scale the data
                    var_data = var_data*curr_scale

                    # Save current year of data to netcdf file
   
                    # Output name
                    fname_out = out_dir+'/'+curr_var+'/'+curr_mod+'/'+curr_var+'.'+experiment_id+'.'+curr_mod+'.'+df1.member_id.iloc[i_ens]+'.' \
                            +np.str(np.int(i_yr[1]))+'.nc'
      
                    # If this file exists, delete it and overwrite
                    if os.path.isfile(fname_out)==True:
                        os.remove(fname_out)
    
                    # Create and open output file
                    #print("WRITING "+fname_out)
                    ncout = netCDF4.Dataset(fname_out,'w',clobber=True,format="NETCDF4_CLASSIC")
                    
                    # Create Dimensions
                    ncout.createDimension('lat',  np.size(lat))
                    ncout.createDimension('lon',  np.size(lon))
                    ncout.createDimension('mon',  np.size(mons))

                    # Create Variables (info: name, precision, dimensions)
                    # dimensions
                    lat_nc   = ncout.createVariable('lat', float, ('lat'), zlib=True)
                    lon_nc   = ncout.createVariable('lon', float, ('lon'), zlib=True)
                    mon_nc   = ncout.createVariable('mon', float, ('mon'), zlib=True)
    
                    # Two-Dimensional Variables
                    varout_nc  = ncout.createVariable(curr_var, float, ('mon','lat','lon'), zlib=True)

                    # Write out dimensional data
                    lat_nc[:]  = lat;         lat_nc.long_name  = 'latitude';  lat_nc.units = 'degrees_north';
                    lon_nc[:]  = lon;         lon_nc.long_name  = 'longitude'; lon_nc.units = 'degrees_east';
                    mon_nc[:]  = mons;        mon_nc.long_name  = 'month';     mon_nc.units = 'unitless';   

                    # Write out data
                    #print("Writing out data")
                    #print("var_data shape is ",var_data.shape)
                    if var_data.shape == varout_nc[:].shape:
                        varout_nc[:] = var_data; varout_nc.long_name = curr_name;  varout_nc.units = curr_units;
                    else:
                        print("Not written: ", fname_out)
                        print("var_data shape is ",var_data.shape)
                        print("varout_nc shape is ",varout_nc[:].shape)
#                     # If time invariant information is available
#                     if df_areacell.empty==False:
#                         areacell_nc    = ncout.createVariable('areacell', float, ('lat','lon'), zlib=True)
#                         areacell_nc[:] = areacell; areacell_nc.long_name = 'grid cell area for atmospheric grid variables';  areacell_nc.units = 'm2';

#                     # For some reason GFDL land mask not the same resolution. Just ignore for now.
#                     #if df_landfrac.empty==False:
#                     if ((df_landfrac.empty==False) & (landfrac.shape[0]==lat.size)):
#                         landfrac_nc    = ncout.createVariable('landfrac', float, ('lat','lon'), zlib=True)
#                         landfrac_nc[:] = landfrac; landfrac_nc.long_name = 'percentage of the grid cell occupied by land (including lakes)';  landfrac_nc.units = 'percent';
            
                    # File Information
                    ncout.comment = 'CMIP6, '+experiment_id+' run. '+curr_mod+': '+df1.member_id.iloc[i_ens]

                # Close the file
                ncout.close()
                

CESM2
rsdt
CESM2:historical:r2i1p1f1:rsdt


In [35]:
def check_data(curr_mod,variable_id,experiment_

curr_mod="CESM2"
df1 = ingrid_cmip6[(ingrid_cmip6.source_id==curr_mod)\
                               & (ingrid_cmip6.variable_id=="rsdt") & \
                               (ingrid_cmip6.experiment_id == experiment_id)]
df1
#print(curr_var)

,activity_id,experiment_id,file_basename,file_dirname,file_fullpath,grid_label,institution_id,member_id,mip_era,source_id,table_id,time_range,variable_id,tracking_id,version
94995,CMIP,historical,rsdt_Amon_CESM2_historical_r6i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r6i1p1f1,CMIP6,CESM2,Amon,185001-201412,rsdt,NaN,v20190308
95060,CMIP,historical,rsdt_Amon_CESM2_historical_r3i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r3i1p1f1,CMIP6,CESM2,Amon,185001-201412,rsdt,NaN,v20190308
95339,CMIP,historical,rsdt_Amon_CESM2_historical_r2i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r2i1p1f1,CMIP6,CESM2,Amon,185001-201412,rsdt,NaN,v20190308
95529,CMIP,historical,rsdt_Amon_CESM2_historical_r4i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r4i1p1f1,CMIP6,CESM2,Amon,185001-201412,rsdt,NaN,v20190308
95572,CMIP,historical,rsdt_Amon_CESM2_historical_r5i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r5i1p1f1,CMIP6,CESM2,Amon,185001-201412,rsdt,NaN,v20190308
97461,CMIP,historical,rsdt_Amon_CESM2_historical_r9i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r9i1p1f1,CMIP6,CESM2,Amon,185001-189912,rsdt,NaN,v20190311
97462,CMIP,historical,rsdt_Amon_CESM2_historical_r9i1p1f1_gn_190001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r9i1p1f1,CMIP6,CESM2,Amon,190001-194912,rsdt,NaN,v20190311
97463,CMIP,historical,rsdt_Amon_CESM2_historical_r9i1p1f1_gn_195001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r9i1p1f1,CMIP6,CESM2,Amon,195001-199912,rsdt,NaN,v20190311
97464,CMIP,historical,rsdt_Amon_CESM2_historical_r9i1p1f1_gn_200001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r9i1p1f1,CMIP6,CESM2,Amon,200001-201412,rsdt,NaN,v20190311
97572,CMIP,historical,rsdt_Amon_CESM2_historical_r8i1p1f1_gn_185001-...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,/m5/haibo/CMIP6mon/CMIP/NCAR/CESM2/historical/...,gn,NCAR,r8i1p1f1,CMIP6,CESM2,Amon,185001-189912,rsdt,NaN,v20190311


In [27]:
curr_mod

'MPI-ESM1-2-HR'

## Save dataframe containing all file information

In [71]:
nc_link
df1.activity_id.iloc[0]

'ScenarioMIP'

In [74]:
nc_link

'http://mary.ldeo.columbia.edu:81/CMIP6/.CMIP/.ScenarioMIP/.BCC/.BCC-CSM2-MR/.ssp245/.r1i1p1f1/.Lmon/.mrsos/.gn/.v20190308/.mrsos_Lmon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc/.mrsos/dods'

In [69]:
http://mary.ldeo.columbia.edu:81/CMIP6/.ScenarioMIP/.BCC/.BCC-CSM2-MR/.ssp245/.r1i1p1f1/.Lmon/.mrsos/.gn/.v20190308/.mrsos_Lmon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc/.mrsos/dods 

0

In [60]:
requests.get(nc_link)  

<Response [404]>

In [61]:
df1

,activity_id,experiment_id,file_basename,file_dirname,file_fullpath,grid_label,institution_id,member_id,mip_era,source_id,table_id,time_range,variable_id,tracking_id,version
74647,ScenarioMIP,ssp245,mrsos_Lmon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_2015...,/m5/haibo/CMIP6mon/ScenarioMIP/BCC/BCC-CSM2-MR...,/m5/haibo/CMIP6mon/ScenarioMIP/BCC/BCC-CSM2-MR...,gn,BCC,r1i1p1f1,CMIP6,BCC-CSM2-MR,Lmon,201501-210012,mrsos,NaN,v20190308


KeyError: 0